In [1]:
import numpy as np
import cv2
import re
import gradio as gr
import pytesseract
from PIL import Image
import pyttsx3

import winsound
from scipy.io.wavfile import read, write
from IPython.display import Audio
from gtts import gTTS 
import os

import googletrans
from googletrans import Translator
from gtts import gTTS 

# Image Scanner

In [2]:
# **Use Gaussian Blurring combined with Adaptive Threshold** 

def blur_and_threshold(gray):
    gray = cv2.GaussianBlur(gray,(3,3),2)
    threshold = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    threshold = cv2.fastNlMeansDenoising(threshold, 11, 31, 9)
    return threshold


# **Find the Biggest Contour** 
def biggest_contour(contours,min_area):
    biggest = None
    max_area = 0
    biggest_n=0
    approx_contour=None
    for n,i in enumerate(contours):
            area = cv2.contourArea(i)
         
            
            if area > min_area/10:
                    peri = cv2.arcLength(i,True)
                    approx = cv2.approxPolyDP(i,0.02*peri,True)
                    if area > max_area and len(approx)==4:
                            biggest = approx
                            max_area = area
                            biggest_n=n
                            approx_contour=approx
                            
                                                   
    return biggest_n,approx_contour


def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    pts=pts.reshape(4,2)
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect


# Find the exact (x,y) coordinates of the biggest contour and crop it out


def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
   

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped


# # Transformation the image

# **1. Convert the image to grayscale**

# **2. Remove noise and smoothen out the image by applying blurring and thresholding techniques**

# **3. Use Canny Edge Detection to find the edges**

# **4. Find the biggest contour and crop it out**


def transformation(image):
    image=image.copy()  
    height, width, channels = image.shape
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image_size=gray.size
    threshold=blur_and_threshold(gray)
    edges = cv2.Canny(threshold,50,150,apertureSize = 7)
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    simplified_contours = []
    for cnt in contours:
        hull = cv2.convexHull(cnt)
        simplified_contours.append(cv2.approxPolyDP(hull,
                                0.001*cv2.arcLength(hull,True),True))
    simplified_contours = np.array(simplified_contours)
    biggest_n,approx_contour = biggest_contour(simplified_contours,image_size)

    threshold = cv2.drawContours(image, simplified_contours ,biggest_n, (0,255,0), 1)

    dst = 0
    if approx_contour is not None and len(approx_contour)==4:
        approx_contour=np.float32(approx_contour)
        dst=four_point_transform(threshold,approx_contour)
    else:
        print("no contour found")
    croppedImage = dst
    return croppedImage


# **Increase the brightness of the image by playing with the "V" value (from HSV)**

def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    
    #random_br = np.random.uniform(0.5,2.0)
   # mask = hsv[:,:,2] * random_br > 255
   # v = np.where(mask, 255, hsv[:,:,2] * random_br)
   # hsv[:,:,2] = v
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img  
    


# **Sharpen the image using Kernel Sharpening Technique**


def final_image(rotated):
    kernel_sharpening = np.array([[0,-1,0], 
                                [-1, 5,-1],
                                [0,-1,0]])
    sharpened = cv2.filter2D(rotated, -1, kernel_sharpening)
    sharpened=increase_brightness(sharpened,30)  
    return sharpened

def imageScanner(image):
    blurred_threshold = transformation(image)
    cleaned_image = final_image(blurred_threshold)
    Final_Image=cv2.imwrite("gradio_output.png", cleaned_image)
    return cleaned_image

# Image to Text

In [3]:
def image_to_text(image):
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    text=pytesseract.image_to_string(image)
    return text

# Text to Speech

In [4]:
def text_to_speech(image,language):
    image_text=image_to_text(image)
    if image_text!= None:
        translator = Translator()
        translated_text = translator.translate(image_text, src='en', dest = language)
        speech = gTTS(translated_text.text, lang = language, slow = False)
        file=speech.save(language+".mp3")
        #os.system("start "+language+".mp3")
        return file 
    
def play(image):
    lang='en'
    text_to_speech(image, lang)

In [5]:
def image_processor(image, options):
    if options=="Scan":
        return imageScanner(image)
    if options=="Text Extraction":
        return image_to_text(image)
    if options=="Convert to Audio":
        return play(image)
    
    
def input_function(image, options):
    prediction=image_processor(image, options)
    return prediction

# GUI

In [6]:
iface = gr.Interface(
  fn=input_function, 
  inputs=["image",gr.inputs.Radio(["Scan", "Text Extraction", "Convert to Audio"])],
  outputs=["image","text",gr.outputs.Audio(label="Output Audio", type="file")],
  title="Image Editor")
iface.launch()

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7860/', None)

<ipython-input-2-19cccbc896be>:125: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  simplified_contours = np.array(simplified_contours)
[2021-08-12 08:54:22,922] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "C:\Users\Lab1\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Lab1\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Lab1\anaconda3\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\Lab1\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_use